## Q1:

## i. Define the set, index, parameters, and decision variables.

### set:
$I$ : set of ingredients, $I=$ {BREAD, PEANUT BUTTER, STRAWBERRY JELLY, GRAHAM CRACKER, MILK, JUICE}
### index:
$i$ : index of ingredients $i$, $i$ $\in$ $I$
### parameters:
$C_i$ : cost of each unit ingredient $i$, $\forall$$i$ $\in$ $I$

$K^F_i$ : calories from fat of each unit ingredient $i$, $\forall$$i$ $\in$ $I$

$K^T_i$ : total calories of each unit ingredient $i$, $\forall$$i$ $\in$ $I$

$V_i$ : vitamin C of each unit ingredient $i$, $\forall$$i$ $\in$ $I$

$P_i$ : protein of each unit ingredient $i$, $\forall$$i$ $\in$ $I$

### decision variables:
$x_i$ : amount of each ingredient $i$ in a meal, $\forall$$i$ $\in$ $I$


## ii. Formulate a linear programming (LP) model with objective function and constraints

### objective function:
$$min = \sum_{i\in I}C_i x_i$$
### constraints:
$$\sum_{i\in I}K^T_i x_i \leq 600$$

$$\sum_{i\in I}K^T_i x_i \geq 400$$

$$\sum_{i\in I}K^F_i x_i \leq  0.3\sum_{i\in I}K^T_i x_i$$

$$\sum_{i\in I}V_i x_i \geq 60$$

$$\sum_{i\in I}P_i x_i \geq 12$$

$$x_{BREAD} = 2$$

$$x_{PEANUT  BUTTER} \geq 2  x_{STRAWBERRY  JELLY}$$

$$x_{MILK} + x_{JUICE} \geq 1$$

$$x_i \geq 0, i \in I$$


### iii. Solve this model by the Python+Pulp solver. You may describe the code in the report.

In [1]:
from pulp import *

In [ ]:
## add parameters
Ingredients = ['BREAD', 'PEANUT BUTTER', 'JELLY', 'CRACKER', 'MILK', 'JUICE']

costs = { # cents
    Ingredients[0]: 5, 
    Ingredients[1]: 4, 
    Ingredients[2]: 7, 
    Ingredients[3]: 8, 
    Ingredients[4]: 15, 
    Ingredients[5]: 35}

fat_kcals = { # kcal
    Ingredients[0]: 10, 
    Ingredients[1]: 75, 
    Ingredients[2]: 0, 
    Ingredients[3]: 20, 
    Ingredients[4]: 70, 
    Ingredients[5]: 0}

total_kcals = { # kcal
    Ingredients[0]: 70, 
    Ingredients[1]: 100, 
    Ingredients[2]: 50, 
    Ingredients[3]: 60, 
    Ingredients[4]: 150, 
    Ingredients[5]: 100}

vit_C = { # mg
    Ingredients[0]: 0, 
    Ingredients[1]: 0, 
    Ingredients[2]: 3, 
    Ingredients[3]: 0, 
    Ingredients[4]: 2, 
    Ingredients[5]: 120}

protein = { # g
    Ingredients[0]: 3, 
    Ingredients[1]: 4, 
    Ingredients[2]: 0, 
    Ingredients[3]: 1, 
    Ingredients[4]: 8, 
    Ingredients[5]: 1}


In [3]:
## model
prob = LpProblem("lunch cost down", LpMinimize)


In [4]:
## decision vars 假設食物不需要整數解
ingredient_vars = LpVariable.dicts(name="Ingredient", indexs=Ingredients, lowBound=0, upBound=None)

In [5]:
## objective function
prob += lpSum([costs[i] * ingredient_vars[i] for i in Ingredients])

In [6]:
## constraints, nutritional requirements
prob += lpSum([total_kcals[i] * ingredient_vars[i] for i in Ingredients]) >= 400 # total kcal
prob += lpSum([total_kcals[i] * ingredient_vars[i] for i in Ingredients]) <= 600 # total kcal
prob += lpSum([fat_kcals[i] * ingredient_vars[i] for i in Ingredients]) <= 0.3 * lpSum([total_kcals[i] * ingredient_vars[i] for i in Ingredients]) # proportion of fat kcal
prob += lpSum([vit_C[i] * ingredient_vars[i] for i in Ingredients]) >= 60 # at least 60 mg vitamin C
prob += lpSum([protein[i] * ingredient_vars[i] for i in Ingredients]) >= 12 # at least 12 g protein
prob += ingredient_vars["BREAD"] == 2 # 2 slices of bread
prob += ingredient_vars["PEANUT BUTTER"] >= 2 * ingredient_vars["JELLY"]  # peanut butter is two times of jelly
prob += ingredient_vars["MILK"] + ingredient_vars["JUICE"] >= 1 # at least one cup of liquid

In [7]:
## solve
prob.solve()

1

### iv. Show the solution results. What’s the managerial implication or insight?

In [8]:
#查看目前解的狀態
print("Status:", LpStatus[prob.status])

#印出解及目標值
for v in prob.variables():
    print(v.name, "=", v.varValue)
print('obj=',value(prob.objective))
#解的另一種方式
# for i in Ingredients:
#   print(ingredient_vars[i],"=",ingredient_vars[i].value())

Status: Optimal
Ingredient_BREAD = 2.0
Ingredient_CRACKER = 1.0394518
Ingredient_JELLY = 0.28737542
Ingredient_JUICE = 0.48421927
Ingredient_MILK = 0.51578073
Ingredient_PEANUT_BUTTER = 0.57475083
obj= 47.310631060000006


### PEANUT_BUTTER 所攝取的脂肪卡路里偏高，如果在重新決策的話可以考慮減少與草莓果醬的限制，讓食物可以更均衡

# Q2

## i. What is the Markovian property?

### The Markovian property is that the series has a finite number of states, stationary transition probabilities and the initial probabilities for all variables are known.

## ii. What is the probability that the unit will be inoperable (because both components are down) after n periods, for n = 2, 5, 10, 20?

In [10]:
import numpy as np

In [11]:
p = np.array([[0.8, 0.2, 0, 0], [0, 0, 0.2, 0.8], [0, 1, 0, 0], [0.8, 0.2, 0, 0]])
print(p)

[[0.8 0.2 0.  0. ]
 [0.  0.  0.2 0.8]
 [0.  1.  0.  0. ]
 [0.8 0.2 0.  0. ]]


In [12]:
p2 = np.dot(p, p)
print(p2)
print(p2[0][2])

[[0.64 0.16 0.04 0.16]
 [0.64 0.36 0.   0.  ]
 [0.   0.   0.2  0.8 ]
 [0.64 0.16 0.04 0.16]]
0.04000000000000001


In [13]:
p4 = np.dot(p2, p2)
p5 = np.dot(p4, p)
print(p5)
print(p5[0][2])

[[0.61952 0.19488 0.03712 0.14848]
 [0.59392 0.17408 0.0464  0.1856 ]
 [0.64    0.232   0.0256  0.1024 ]
 [0.61952 0.19488 0.03712 0.14848]]
0.037120000000000014


In [14]:
p10 = np.dot(p5, p5)
print(p10)
print(p10[0][2])

[[0.61529129 0.19220439 0.03850086 0.15400346]
 [0.61601382 0.1929815  0.03820093 0.15280374]
 [0.61410509 0.19100467 0.03897805 0.15591219]
 [0.61529129 0.19220439 0.03850086 0.15400346]]
0.03850086400000004


In [15]:
p20 = np.dot(p10, p10)
print(p20)
print(p20[0][2])

[[0.61538449 0.19230756 0.03846159 0.15384635]
 [0.61538541 0.19230853 0.03846121 0.15384485]
 [0.61538306 0.19230606 0.03846218 0.1538487 ]
 [0.61538449 0.19230756 0.03846159 0.15384635]]
0.03846158821035613


#### probability that the unit will be inoperable after 2 periods is: 0.04
#### probability that the unit will be inoperable after 5 periods is: 0.03712
#### probability that the unit will be inoperable after 10 periods is: 0.0385
#### probability that the unit will be inoperable after 20 periods is: 0.03846

## iii. What are the steady state probabilities of the state of this Markov chain?

In [16]:
p40 = np.dot(p20, p20)
print(p40)
print(p40[0][2])

[[0.61538462 0.19230769 0.03846154 0.15384615]
 [0.61538462 0.19230769 0.03846154 0.15384615]
 [0.61538462 0.19230769 0.03846154 0.15384615]
 [0.61538462 0.19230769 0.03846154 0.15384615]]
0.038461538461615784


#### the steady state probabilities of the state of this Markov chain is: 
[$0.61538462,$  $0.19230769,$  $0.03846154,$  $0.15384615$]

## iv. If it costs 30,000 per period when the unit is inoperable (both components down) and zero otherwise, what is the (long run) expected average cost per period?

In [17]:
inoperable_cost = 30000
p_inoperable = p40[0][2] 
expected_cost = p_inoperable * inoperable_cost + (1 - p_inoperable) * 0
print("expected_cost is: ", expected_cost)

expected_cost is:  1153.8461538484735
